## Motif price generator - for items in inventory
This code parses TTC item and price tables and computes prices per motif

Copy SLPP.py here for ease of use, source is on github SLPP parses the Lua stuff and gives us python data structures

In [21]:
import pandas as pd
from collections import Counter

In [22]:
pd.set_option('display.max_rows', None)

### SLPP code pasted below

In [23]:
import re
import sys
from numbers import Number

import six

ERRORS = {
    'unexp_end_string': u'Unexpected end of string while parsing Lua string.',
    'unexp_end_table': u'Unexpected end of table while parsing Lua string.',
    'mfnumber_minus': u'Malformed number (no digits after initial minus).',
    'mfnumber_dec_point': u'Malformed number (no digits after decimal point).',
    'mfnumber_sci': u'Malformed number (bad scientific format).',
}

def sequential(lst):
    length = len(lst)
    if length == 0 or lst[0] != 0:
        return False
    for i in range(length):
        if i + 1 < length:
            if lst[i] + 1 != lst[i+1]:
                return False
    return True


class ParseError(Exception):
    pass


class SLPP(object):

    def __init__(self):
        self.text = ''
        self.ch = ''
        self.at = 0
        self.len = 0
        self.depth = 0
        self.space = re.compile('\s', re.M)
        self.alnum = re.compile('\w', re.M)
        self.newline = '\n'
        self.tab = '\t'

    def decode(self, text):
        if not text or not isinstance(text, six.string_types):
            return
        # FIXME: only short comments removed
        reg = re.compile('--.*$', re.M)
        text = reg.sub('', text, 0)
        self.text = text
        self.at, self.ch, self.depth = 0, '', 0
        self.len = len(text)
        self.next_chr()
        result = self.value()
        return result

    def encode(self, obj):
        self.depth = 0
        return self.__encode(obj)

    def __encode(self, obj):
        s = ''
        tab = self.tab
        newline = self.newline

        if isinstance(obj, str):
            s += '"%s"' % obj.replace(r'"', r'\"')
        elif six.PY2 and isinstance(obj, unicode):
            s += '"%s"' % obj.encode('utf-8').replace(r'"', r'\"')
        elif six.PY3 and isinstance(obj, bytes):
            s += '"{}"'.format(''.join(r'\x{:02x}'.format(c) for c in obj))
        elif isinstance(obj, bool):
            s += str(obj).lower()
        elif obj is None:
            s += 'nil'
        elif isinstance(obj, Number):
            s += str(obj)
        elif isinstance(obj, (list, tuple, dict)):
            self.depth += 1
            if len(obj) == 0 or (not isinstance(obj, dict) and len([
                    x for x in obj
                    if isinstance(x, Number) or (isinstance(x, six.string_types) and len(x) < 10)
               ]) == len(obj)):
                newline = tab = ''
            dp = tab * self.depth
            s += "%s{%s" % (tab * (self.depth - 2), newline)
            if isinstance(obj, dict):
                key = '[%s]' if all(isinstance(k, (int, long)) for k in obj.keys()) else '%s'
                contents = [dp + (key + ' = %s') % (k, self.__encode(v)) for k, v in obj.items()]
                s += (',%s' % newline).join(contents)
            else:
                s += (',%s' % newline).join(
                    [dp + self.__encode(el) for el in obj])
            self.depth -= 1
            s += "%s%s}" % (newline, tab * self.depth)
        return s

    def white(self):
        while self.ch:
            if self.space.match(self.ch):
                self.next_chr()
            else:
                break

    def next_chr(self):
        if self.at >= self.len:
            self.ch = None
            return None
        self.ch = self.text[self.at]
        self.at += 1
        return True

    def value(self):
        self.white()
        if not self.ch:
            return
        if self.ch == '{':
            return self.object()
        if self.ch == "[":
            self.next_chr()
        if self.ch in ['"',  "'",  '[']:
            return self.string(self.ch)
        if self.ch.isdigit() or self.ch == '-':
            return self.number()
        return self.word()

    def string(self, end=None):
        s = ''
        start = self.ch
        if end == '[':
            end = ']'
        if start in ['"',  "'",  '[']:
            while self.next_chr():
                if self.ch == end:
                    self.next_chr()
                    if start != "[" or self.ch == ']':
                        return s
                if self.ch == '\\' and start == end:
                    self.next_chr()
                    if self.ch != end:
                        s += '\\'
                s += self.ch
        raise ParseError(ERRORS['unexp_end_string'])

    def object(self):
        o = {}
        k = None
        idx = 0
        numeric_keys = False
        self.depth += 1
        self.next_chr()
        self.white()
        if self.ch and self.ch == '}':
            self.depth -= 1
            self.next_chr()
            return o  # Exit here
        else:
            while self.ch:
                self.white()
                if self.ch == '{':
                    o[idx] = self.object()
                    idx += 1
                    continue
                elif self.ch == '}':
                    self.depth -= 1
                    self.next_chr()
                    if k is not None:
                        o[idx] = k
                    if len([key for key in o if isinstance(key, six.string_types + (float,  bool, tuple))]) == 0:
                        so = sorted([key for key in o])
                        if sequential(so):
                            ar = []
                            for key in o:
                                ar.insert(key, o[key])
                            o = ar
                    return o  # or here
                else:
                    if self.ch == ',':
                        self.next_chr()
                        continue
                    else:
                        k = self.value()
                        if self.ch == ']':
                            self.next_chr()
                    self.white()
                    ch = self.ch
                    if ch in ('=', ','):
                        self.next_chr()
                        self.white()
                        if ch == '=':
                            o[k] = self.value()
                        else:
                            o[idx] = k
                        idx += 1
                        k = None
        raise ParseError(ERRORS['unexp_end_table'])  # Bad exit here

    words = {'true': True, 'false': False, 'nil': None}
    def word(self):
        s = ''
        if self.ch != '\n':
            s = self.ch
        self.next_chr()
        while self.ch is not None and self.alnum.match(self.ch) and s not in self.words:
            s += self.ch
            self.next_chr()
        return self.words.get(s, s)

    def number(self):
        def next_digit(err):
            n = self.ch
            self.next_chr()
            if not self.ch or not self.ch.isdigit():
                raise ParseError(err)
            return n
        n = ''
        try:
            if self.ch == '-':
                n += next_digit(ERRORS['mfnumber_minus'])
            n += self.digit()
            if n == '0' and self.ch in ['x', 'X']:
                n += self.ch
                self.next_chr()
                n += self.hex()
            else:
                if self.ch and self.ch == '.':
                    n += next_digit(ERRORS['mfnumber_dec_point'])
                    n += self.digit()
                if self.ch and self.ch in ['e', 'E']:
                    n += self.ch
                    self.next_chr()
                    if not self.ch or self.ch not in ('+', '-'):
                        raise ParseError(ERRORS['mfnumber_sci'])
                    n += next_digit(ERRORS['mfnumber_sci'])
                    n += self.digit()
        except ParseError:
            t, e = sys.exc_info()[:2]
            print(e)
            return 0
        try:
            return int(n, 0)
        except:
            pass
        return float(n)

    def digit(self):
        n = ''
        while self.ch and self.ch.isdigit():
            n += self.ch
            self.next_chr()
        return n

    def hex(self):
        n = ''
        while self.ch and (self.ch in 'ABCDEFabcdef' or self.ch.isdigit()):
            n += self.ch
            self.next_chr()
        return n


slpp = SLPP()

__all__ = ['slpp']

### Load InventoryInsight Data

In [24]:
iia = open(r"C:\Users\jtern\Documents\Elder Scrolls Online\live\SavedVariables\IIfA.lua",'r').read()

In [25]:
settings = slpp.decode(iia.split('IIfA_Settings =')[-1])

In [26]:
data = slpp.decode(iia.split('IIfA_Data =')[-1])

### Find all motifs in our current account/server

In [27]:
accounts = ['@TholosTB', '@TholosTB2','@TholosTB_3']
server = 'NA'
#rex = re.compile ("Crafting Motif (\d+)|Style")
rex = re.compile ("Crafting Motif (\d+)")
#rex = re.compile("(Mother's|Medusa)")
#rex = re.compile ("Opal .* Staff")

In [28]:
motifs = []
item_counter = Counter()
for account in accounts:
    for key,entry in data['Default'][account]['$AccountWide']['Data'][server]['DBv3'].items():
        item_counter[entry['itemName']] +=1
        if rex.search(entry['itemName'] ): 
            print(key,entry)
            for loc in entry['locations'].keys() :
                print (loc)
                charname = settings['Default'][account][loc]['$LastCharacterName'] if re.match('\d+',str(loc)) and loc in settings['Default'][account] else str(loc)
                locref = entry['locations'][loc]
                #qty = list(locref['bagSlot'].items())[0][1]
                motifs.append({'item' : entry['itemName'],
                               'character' : f'{account}:{charname}',
                               'quality' : entry['itemQuality'],
                               'locations' : list(locref.items())
                               #,
                              #'qty' : qty
                              })

121333 {'filterType': 3, 'itemQuality': 4, 'itemName': 'Crafting Motif 50: Telvanni Axes', 'itemLink': '|H1:item:121333:5:1:0:0:0:0:0:0:0:0:0:0:0:0:0:0:0:0:0:0|h|h', 'locations': {'8796093060351489': {'bagSlot': {85: 1}, 'bagID': 1}}}
8796093060351489
71691 {'filterType': 3, 'itemQuality': 4, 'itemName': 'Crafting Motif 25: Aldmeri Dominion Boots', 'itemLink': '|H1:item:71691:5:1:0:0:0:0:0:0:0:0:0:0:0:0:0:0:0:0:0:0|h|h', 'locations': {'8796093060351489': {'bagSlot': {38: 2}, 'bagID': 1}}}
8796093060351489
156618 {'filterType': 3, 'itemQuality': 4, 'itemName': 'Crafting Motif 81: New Moon Priest Maces', 'itemLink': '|H1:item:156618:5:1:0:0:0:0:0:0:0:0:0:0:0:0:0:0:0:0:0:0|h|h', 'locations': {'8796093060351489': {'bagSlot': {134: 1}, 'bagID': 1}}}
8796093060351489
64729 {'filterType': 3, 'itemQuality': 4, 'itemName': 'Crafting Motif 19: Mercenary Swords', 'itemLink': '|H1:item:64729:5:1:0:0:0:0:0:0:0:0:0:0:0:0:0:0:0:0:0:0|h|h', 'locations': {'8796093060351489': {'bagSlot': [1], 'bagID': 1

In [88]:
data['Default'].keys()

dict_keys(['@TholosTB2', '@TholosTB_3', '@TholosTB'])

In [9]:
sorted(motifs,key=lambda x : int(rex.search(x['item']).group(1)))

IndexError: no such group

In [90]:
item_counter.most_common()

[('Sealed Jewelry Crafter Writ', 21),
 ('Sealed Blacksmithing Writ', 13),
 ('Sealed Woodworking Writ', 10),
 ('Sash of Julianos', 10),
 ('Sealed Alchemy Writ', 10),
 ('Shoes of Julianos', 10),
 ('Crown Tri-Restoration Potion', 10),
 ('Gloves of Julianos', 10),
 ('Inferno Staff of Julianos', 10),
 ('Sealed Clothier Writ', 9),
 ('Crown Lethal Poison', 9),
 ('Breeches of Julianos', 9),
 ('Sealed Enchanting Writ', 9),
 ('Crown Repair Kit', 9),
 ("Crafty Alfiq's Ring", 8),
 ('Deep Winter Charity Writ', 8),
 ("Ring of a Mother's Sorrow", 8),
 ('Gold Coast Spellcaster Elixir', 7),
 ('Ring of the Withered Hand', 7),
 ('Crown Experience Scroll', 6),
 ('ruby ash restoration staff^n', 6),
 ('Gold Coast Warrior Elixir', 6),
 ('homespun breeches^p', 6),
 ('Instant All Research', 6),
 ('sip of health', 6),
 ('Ring of the Wild Hunt', 6),
 ("Belt of Hunding's Rage", 6),
 ("Crafty Alfiq's Amulet", 6),
 ('Collar of Bones', 6),
 ("Sword-Singer's Ring", 6),
 ('Gold Coast Draining Poison', 5),
 ("Breeches 

In [91]:
len(motifs)

371

In [29]:
motif_df = pd.DataFrame(motifs)

In [30]:
motif_df['motif_num'] = motif_df['item'].str.extract(rex)

In [12]:
len(motif_df.item.unique())

315

In [13]:
motif_df.sort_values('item')

,item,character,quality,locations,motif_num
192,Crafting Motif 11: Ancient Elf Style,@TholosTB:4674,4,"[(bagID, 8), (bagSlot, {32: 12})]",11
15,Crafting Motif 12: Barbaric Style,@TholosTB:4674,4,"[(bagID, 8), (bagSlot, {33: 8})]",12
302,Crafting Motif 13: Primal Style,@TholosTB:R'af-O,4,"[(bagID, 1), (bagSlot, {28: 1})]",13
349,Crafting Motif 13: Primal Style,@TholosTB_3:Bank,4,"[(bagID, 2), (bagSlot, {13: 1})]",13
303,Crafting Motif 13: Primal Style,@TholosTB:4674,4,"[(bagID, 8), (bagSlot, {37: 16})]",13
304,Crafting Motif 13: Primal Style,@TholosTB:Heals-the-Softskins,4,"[(bagID, 1), (bagSlot, {88: 1})]",13
281,Crafting Motif 14: Daedric Style,@TholosTB:4674,4,"[(bagID, 8), (bagSlot, {39: 8})]",14
347,Crafting Motif 14: Daedric Style,@TholosTB2:Amberlager,4,"[(bagID, 1), (bagSlot, {82: 1})]",14
172,Crafting Motif 15: Dwemer Axes,@TholosTB:Skurge of Cyrodiil,4,"[(bagID, 1), (bagSlot, {6: 1})]",15
324,Crafting Motif 15: Dwemer Belts,@TholosTB:R'af-O,4,"[(bagID, 1), (bagSlot, {37: 1})]",15


In [96]:
motif_df.sort_values('item').groupby(['item','character']).count()

locations  \
item                                             character                                      
Crafting Motif 11: Ancient Elf Style             @TholosTB:4674                             1   
Crafting Motif 12: Barbaric Style                @TholosTB:4674                             1   
Crafting Motif 13: Primal Style                  @TholosTB:4674                             1   
                                                 @TholosTB:R'af-O                           1   
                                                 @TholosTB_3:Bank                           1   
Crafting Motif 14: Daedric Style                 @TholosTB2:Amberlager                      1   
                                                 @TholosTB:4674                             1   
Crafting Motif 15: Dwemer Axes                   @TholosTB:Skurge of Cyrodiil               1   
Crafting Motif 15: Dwemer Belts                  @TholosTB:Bank                             1   
Crafting Motif 15: Dwemer Boots                  @TholosTB:Skurge of Cyrodiil               1   
Crafting Motif 15: Dwemer Bows                   @TholosTB:Bank                             1   
Crafting Motif 15: Dwemer Chests                 @TholosTB:Skurge of Cyrodiil               1   
                                                 @TholosTB:Tarrant of Merentha              1   
Crafting Motif 15: Dwemer Daggers                @TholosTB:Skurge of Cyrodiil               1   
Crafting Motif 15: Dwemer Gloves                 @TholosTB:Bank                             1   
                                                 @TholosTB:Tholos Greymane                  1   
Crafting Motif 15: Dwemer Helmets                @TholosTB:Skurge of Cyrodiil               1   
Crafting Motif 15: Dwemer Legs                   @TholosTB:Skurge of Cyrodiil               1   
Crafting Motif 15: Dwemer Shoulders              @TholosTB:Skurge of Cyrodiil               1   
Crafting Motif 15: Dwemer Staves                 @TholosTB:R'af-O                           1   
Crafting Motif 16: Glass Chests                  @TholosTB:Tarrant of Merentha              1   
Crafting Motif 16: Glass Staves                  @TholosTB:Bank                             1   
Crafting Motif 17: Xivkyn Belts                  @TholosTB:Skurge of Cyrodiil               1   
Crafting Motif 17: Xivkyn Bows                   @TholosTB:Skurge of Cyrodiil               1   
Crafting Motif 17: Xivkyn Legs                   @TholosTB:Skurge of Cyrodiil               1   
Crafting Motif 17: Xivkyn Shields                @TholosTB:Tarrant of Merentha              1   
Crafting Motif 17: Xivkyn Staves                 @TholosTB:Tarrant of Merentha              1   
Crafting Motif 18: Akaviri Maces                 @TholosTB:Tarrant of Merentha              1   
Crafting Motif 18: Akaviri Shoulders             @TholosTB:Bank                             1   
Crafting Motif 19: Mercenary Axes                @TholosTB:Bank                             1   
                                                 @TholosTB:Skurge of Cyrodiil               1   
Crafting Motif 19: Mercenary Belts               @TholosTB:R'af-O                           1   
Crafting Motif 19: Mercenary Daggers             @TholosTB:Bank                             1   
Crafting Motif 19: Mercenary Shoulders           @TholosTB:Tarrant of Merentha              1   
Crafting Motif 19: Mercenary Swords              @TholosTB:Bank                             1   
Crafting Motif 1: High Elf Style                 @TholosTB:4674                             1   
Crafting Motif 20: Yokudan Axes                  @TholosTB:Skurge of Cyrodiil               1   
Crafting Motif 20: Yokudan Legs                  @TholosTB:Heals-the-Softskins              1   
                                                 @TholosTB:Skurge of Cyrodiil               1   
Crafting Motif 20: Yokudan Maces                 @TholosTB:Bank                             1   
Crafting M

### Load TTC price and item tables

In [97]:
item_prices = open(r"C:\Users\jtern\Documents\Elder Scrolls Online\live\AddOns\TamrielTradeCentre\PriceTable.lua",'r').read()

In [98]:
prices = slpp.decode(item_prices.split('self.PriceTable=')[-1])

In [99]:
item_lookup = open(r"C:\Users\jtern\Documents\Elder Scrolls Online\live\AddOns\TamrielTradeCentre\ItemLookUpTable_EN.lua",'r').read()

In [100]:
items = slpp.decode(item_lookup.split('self.ItemLookUpTable=')[-1])

### Get all motif keys from TTC items table for motifs in our inventory

In [101]:
motif_keys = [key for key in items.keys() for m in motifs if key.lower() == m['item'].lower()]

In [102]:
motif_keys

['crafting motif 17: xivkyn belts',
 'crafting motif 15: dwemer bows',
 'crafting motif 19: mercenary axes',
 'crafting motif 19: mercenary axes',
 'crafting motif 12: barbaric style',
 'crafting motif 14: daedric style',
 'crafting motif 14: daedric style',
 'crafting motif 19: mercenary swords',
 'crafting motif 15: dwemer staves',
 'crafting motif 18: akaviri maces',
 'crafting motif 17: xivkyn bows',
 'crafting motif 15: dwemer axes',
 'crafting motif 13: primal style',
 'crafting motif 13: primal style',
 'crafting motif 13: primal style',
 'crafting motif 15: dwemer daggers',
 'crafting motif 19: mercenary belts',
 'crafting motif 15: dwemer helmets',
 'crafting motif 18: akaviri shoulders',
 'crafting motif 15: dwemer gloves',
 'crafting motif 15: dwemer gloves',
 'crafting motif 6: redguard style',
 'crafting motif 11: ancient elf style',
 'crafting motif 19: mercenary daggers',
 'crafting motif 2: dark elf style',
 'crafting motif 15: dwemer shoulders',
 'crafting motif 17: xi

### Iterate over all the motifs and build a new list with prices and names. If prices are not found, leave a blank entry


In [103]:

data = []
for motif_key in motif_keys:
    item = items[motif_key]
    entry = {'motif' : motif_key}
    entry['item_id'] = list(item.items())[0][1]
    try:
        price = prices['Data'][entry['item_id']]
    except KeyError:
        data.append(entry)
        continue
    qual = list(price.keys())[0]
    level_dict=price[qual]
    level = list(level_dict.keys())[0]
    traits_dict = level_dict[level]
    traits = list(traits_dict.keys())[0]
    prices_dict = traits_dict[traits]
    entry = {**entry,**prices_dict}
    data.append(entry)

In [104]:
motifs_in_inventory_df = pd.DataFrame(data)

In [105]:
motifs_in_inventory_df['motif_num'] = motifs_in_inventory_df['motif'].str.extract('crafting motif (\d+)').astype(int)

In [106]:
motifs_in_inventory_df.sort_values('SuggestedPrice')

,motif,item_id,Avg,Max,Min,EntryCount,AmountCount,SuggestedPrice,motif_num
28,crafting motif 5: breton style,6534,354.49,22000,1.0,612,639,71.75,5
21,crafting motif 6: redguard style,6500,628.81,100000,1.0,596,618,72.07,6
24,crafting motif 2: dark elf style,6511,492.28,20000,1.0,621,661,75.99,2
35,crafting motif 3: wood elf style,6580,349.97,20000,1.0,631,644,77.40,3
34,crafting motif 7: khajiit style,6573,415.72,20000,2.5,618,633,79.82,7
27,crafting motif 1: high elf style,6533,604.02,50000,1.0,681,701,82.98,1
39,crafting motif 9: argonian style,6612,406.87,25000,1.0,616,643,83.49,9
243,crafting motif 63: dremora daggers,18738,1736.84,55039,1.0,566,632,276.41,63
244,crafting motif 63: dremora daggers,18738,1736.84,55039,1.0,566,632,276.41,63
251,crafting motif 63: dremora gloves,18760,2032.48,40000,1.0,424,464,286.14,63


In [108]:
motifs_in_inventory_df.loc[motifs_in_inventory_df['SuggestedPrice']<40000].sort_values('motif_num')['motif'].unique()

array(['crafting motif 1: high elf style',
       'crafting motif 2: dark elf style',
       'crafting motif 3: wood elf style',
       'crafting motif 5: breton style',
       'crafting motif 6: redguard style',
       'crafting motif 7: khajiit style',
       'crafting motif 9: argonian style',
       'crafting motif 11: ancient elf style',
       'crafting motif 12: barbaric style',
       'crafting motif 13: primal style',
       'crafting motif 14: daedric style',
       'crafting motif 15: dwemer shoulders',
       'crafting motif 15: dwemer legs',
       'crafting motif 15: dwemer boots',
       'crafting motif 15: dwemer chests',
       'crafting motif 15: dwemer gloves',
       'crafting motif 15: dwemer helmets',
       'crafting motif 15: dwemer daggers',
       'crafting motif 15: dwemer axes',
       'crafting motif 15: dwemer staves',
       'crafting motif 15: dwemer belts',
       'crafting motif 15: dwemer bows',
       'crafting motif 16: glass chests',
       'crafti

In [109]:
sorted([m for m in motifs if not re.search('21|45|35|42',m)],key=lambda x : int(rex.search(x).group(1)))

TypeError: expected string or bytes-like object

In [110]:
motifs_df2 = motifs_in_inventory_df.merge(motif_df)

ValueError: You are trying to merge on int32 and object columns. If you wish to proceed you should use pd.concat

In [16]:
savvy = ["crafting motif 41: celestial boots",
"crafting motif 47: buoyant armiger daggers",
"crafting motif 47: buoyant armiger swords"]


In [17]:
motif_df.loc[motif_df['item'].str.lower().isin(savvy)].sort_values('character')

,item,character,quality,locations,motif_num
30,Crafting Motif 47: Buoyant Armiger Daggers,@TholosTB:R'af-O,4,"[(bagID, 1), (bagSlot, {65: 2})]",47
43,Crafting Motif 47: Buoyant Armiger Swords,@TholosTB:Skurge of Cyrodiil,4,"[(bagID, 1), (bagSlot, {60: 1})]",47
105,Crafting Motif 41: Celestial Boots,@TholosTB:Skurge of Cyrodiil,4,"[(bagID, 1), (bagSlot, {33: 1})]",41


In [113]:
motif_df.loc[motif_df['item'].str.lower().isin(savvy)].sort_values('item')

,item,character,locations,motif_num
342,Crafting Motif 19: Mercenary Belts,@TholosTB:R'af-O,"[(bagID, 1), (bagSlot, {37: 1})]",19
346,Crafting Motif 19: Mercenary Daggers,@TholosTB:Bank,"[(bagID, 6), (bagSlot, {217: 1})]",19
110,Crafting Motif 20: Yokudan Axes,@TholosTB:Skurge of Cyrodiil,"[(bagID, 1), (bagSlot, {77: 1})]",20
314,Crafting Motif 20: Yokudan Maces,@TholosTB:Bank,"[(bagID, 6), (bagSlot, {219: 1})]",20
105,Crafting Motif 20: Yokudan Shoulders,@TholosTB:R'af-O,"[(bagID, 1), (bagSlot, {41: 1})]",20
178,Crafting Motif 20: Yokudan Swords,@TholosTB:Bank,"[(bagID, 6), (bagSlot, {220: 1})]",20
272,Crafting Motif 27: Ebonheart Pact Daggers,@TholosTB:Skurge of Cyrodiil,"[(bagID, 1), (bagSlot, {82: 1})]",27
181,Crafting Motif 27: Ebonheart Pact Maces,@TholosTB:Skurge of Cyrodiil,"[(bagID, 1), (bagSlot, [1])]",27
66,Crafting Motif 27: Ebonheart Pact Swords,@TholosTB:Bank,"[(bagID, 6), (bagSlot, {221: 1})]",27
215,Crafting Motif 52: Redoran Belts,@TholosTB:Skurge of Cyrodiil,"[(bagID, 1), (bagSlot, {67: 1})]",52


In [5]:
import base64
import struct

In [8]:
s = "CSSK29ф////+AAP/+43f/9f//b/7/////////////////////kNh///////+gAH///////7Ebf/9AAP//gAABJAABezb//whA//9KAP///////wAAP5d//73rf/+AEH////+t/T+/KUhSkf/+AhAABEAQAAH////+AAAAAf/+huwEAf/+AAAAAAZwAAAAAAAAwNwAAAAAAAAAAAAAAAAAAAAAAAAAAAAA1"
struct.unpack('HhL',bytearray(s,encoding='utf-8'))

error: unpack requires a buffer of 8 bytes

In [3]:
bytearray(s,encoding='utf-8')

bytearray(b'CSSK29\xd1\x84////+AAP/+43f/9f//b/7/////////////////////kNh///////+gAH///////7Ebf/9AAP//gAABJAABezb//whA//9KAP///////wAAP5d//73rf/+AEH////+t/T+/KUhSkf/+AhAABEAQAAH////+AAAAAf/+huwEAf/+AAAAAAZwAAAAAAAAwNwAAAAAAAAAAAAAAAAAAAAAAAAAAAAA1')

In [10]:
researched = {'From-Cool-Springs':"CSCR29ф//////////////////////////////////////////////////////0",
            "Soloh'T-dar" : "CSCR29фxkSkgOAhkAgCyFIASSI0ADQyKSMLSSNRiiggAQAAAAUzcBWqlUC6nU0" }
styles = {'From-Cool-Springs' : "CSSK29ф////+AAP/+43//9f//b/7/////////////////////kNh///////+gAH///////7Ebf/9AAP//gAABJAABezb//whA//9KAP///////wAAP5d//73rf/+AEH////+t/T+/KUhSkf/+AhAABEAQAAH////+AAAAAf/+huwEAf/+AAAAAIb0AAAAAAAAwNyiAAAAAAAAAAAAAAAAAAAAAAAAAAAA1",
          "Soloh'T-dar" : "CSSK29фtdAAAAAAAAAALXQAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA1",
          "Raf'O" : "CSSK29фAQAAAAAAAAAAAEAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA1"
}

In [11]:
for name,style_str in styles.items():
    print(name,len(style_str))

From-Cool-Springs 226
Soloh'T-dar 226
Raf'O 226


In [17]:
''.join(format(i,'08b') for i in bytearray(styles["From-Cool-Springs"][7:],encoding='utf-8'))

'001011110010111100101111001011110010101101000001010000010101000000101111001010110011010000110011001011110010111100111001011001100010111100101111011000100010111100110111001011110010111100101111001011110010111100101111001011110010111100101111001011110010111100101111001011110010111100101111001011110010111100101111001011110010111100101111011010110100111001101000001011110010111100101111001011110010111100101111001011110010101101100111010000010100100000101111001011110010111100101111001011110010111100101111001101110100010101100010011001100010111100111001010000010100000101010000001011110010111101100111010000010100000101000010010010100100000101000001010000100110010101111010011000100010111100101111011101110110100001000001001011110010111100111001010010110100000101010000001011110010111100101111001011110010111100101111001011110111011101000001010000010101000000110101011001000010111100101111001101110011001101110010011001100010111100101011010000010100010101001000001011110010111100101111001011110010101

In [16]:
styles['From-Cool-Springs'][6:]

'ф////+AAP/+43//9f//b/7/////////////////////kNh///////+gAH///////7Ebf/9AAP//gAABJAABezb//whA//9KAP///////wAAP5d//73rf/+AEH////+t/T+/KUhSkf/+AhAABEAQAAH////+AAAAAf/+huwEAf/+AAAAAIb0AAAAAAAAwNyiAAAAAAAAAAAAAAAAAAAAAAAAAAAA1'

In [18]:
len(styles['From-Cool-Springs'][6:])

220

15.714285714285714

In [60]:
motif_df.loc[motif_df.item.str.contains('Stags')]

,item,character,quality,locations,motif_num
200,Crafting Motif 77: Stags of Z'en Axes,@TholosTB:Heals-the-Softskins,4,"[(bagSlot, {53: 1}), (bagID, 1)]",77
